    1- Key 64 or 56 becomes KeyL and KeyR 28 each
    2- circular shift depending on round
    3- take circulated for round, take uncirculated for next round




In [86]:
import binascii
import numpy as np
import keyTables, Sbox, textTables, additionalFunction, functions

roundNumber = 1
C = [] #KeyLeft
D = [] #KeyRight
L = [] #PlainLeft
R = [] #PlainRight
temp = []
roundKey =[]

RoundShift= [1,1,2,2,2,2,2,2,1,2,2,2,2,2,2,1]


In [74]:


#Key
permutationChoice1 = [57, 49, 41, 33, 25, 17, 9,
                    1, 58, 50, 42, 34, 26, 18,
                    10, 2, 59, 51, 43, 35, 27,
                    19, 11, 3, 60, 52, 44, 36,
                    63, 55, 47, 39, 31, 23, 15,
                    7, 62, 54, 46, 38, 30, 22,
                    14, 6, 61, 53, 45, 37, 29,
                    21, 13, 5, 28, 20, 12, 4]
                    
permutationChoice2 = [14, 17, 11, 24, 1, 5, 3, 28,
                    15, 6, 21, 10, 23, 19, 12, 4,
                    26, 8, 16, 7, 27, 20, 13, 2,
                    41, 52, 31, 37, 47, 55, 30, 40,
                    51, 45, 33, 48, 44, 49, 39, 56,
                    34, 53, 46, 42, 50, 36, 29, 32]


In [75]:
K= [0,0,0,0,0,0,0,1,  0,0,1,0,0,0,1,1,  0,1,0,0,0,1,0,1,  0,1,1,0,0,1,1,1,  1,0,0,0,1,0,0,1,  1,0,1,0,1,0,1,1,  1,1,0,0,1,1,0,1,  1,1,1,0,1,1,1,1]
plainText = [0,0,0,0,0,0,0,1,  0,0,1,0,0,0,1,1,  0,1,0,0,0,1,0,1,  0,1,1,0,0,1,1,1,  1,0,0,0,1,0,0,1,  1,0,1,0,1,0,1,1,  1,1,0,0,1,1,0,1,  1,1,1,0,1,1,1,1]

In [76]:
def PC(Key, pcNumber): #PermutationChoice
    permutationChoice = []
    if(pcNumber == 1):
        permutationChoice = permutationChoice1
    else:
        permutationChoice = permutationChoice2


    arr = []
    for i in range(len(permutationChoice)):
       arr.append(Key[permutationChoice[i] -1])
    # for i in range(0, len(arr),8):
    #     print(arr[i],arr[i+1],arr[i+2],arr[i+3],arr[i+4],arr[i+5],arr[i+6],arr[i+7])
    return arr

def IP(Plaintext): #InitialPermutation
    
    arr = []
    for i in range(len(initialPermutation)):
       arr.append(Plaintext[initialPermutation[i] -1])
    # for i in range(0, len(arr),8):
    #     print(arr[i],arr[i+1],arr[i+2],arr[i+3],arr[i+4],arr[i+5],arr[i+6],arr[i+7])
    return arr
def EP(rightPlaintext): #ExpansionPermutation
    
    arr = []
    for i in range(len(expansionPermutation)):
       arr.append(rightPlaintext[expansionPermutation[i] -1])
    # for i in range(0, len(arr),6):
    #     print(arr[i],arr[i+1],arr[i+2],arr[i+3],arr[i+4],arr[i+5])
    return arr

def PF(Sbox): #PermutationFunction
    
    arr = []
    for i in range(len(permutationFunction)):
       arr.append(Sbox[permutationFunction[i] -1])
    # for i in range(0, len(arr),8):
    #     print(arr[i],arr[i+1],arr[i+2],arr[i+3],arr[i+4],arr[i+5],arr[i+6],arr[i+7])
    return arr
def inverseIP(Plaintext): #PermutationFunction
    
    arr = []
    for i in range(len(inverseInitialPermutation)):
       arr.append(Plaintext[inverseInitialPermutation[i] -1])
    for i in range(0, len(arr),8):
        print(arr[i],arr[i+1],arr[i+2],arr[i+3],arr[i+4],arr[i+5],arr[i+6],arr[i+7])
    return arr

In [77]:
def Sbox(ExorK):
    arr = []
    row = 0
    column = 0
    for i in range(0,len(ExorK),6):
        row = int(f"{ExpansionXorKey[i]}{ExpansionXorKey[i+5]}",2)
        column = int(f"{ExpansionXorKey[i+1]}{ExpansionXorKey[i+2]}{ExpansionXorKey[i+3]}{ExpansionXorKey[i+4]}",2)
        value = SBox[f"S{int(i/6 + 1)}"][row][column]
        # print(f"S{int(i/6 + 1)}")
        # print(f"row = {row} \t column = {column} \t value = {value}")
        temp= bin(value)
        binaryNumber = temp[2:]
        binaryNumber = list(map(int, list(binaryNumber)))
        if(len(binaryNumber)==4):
            arr.append(binaryNumber[0])
            arr.append(binaryNumber[1])
            arr.append(binaryNumber[2])
            arr.append(binaryNumber[3])
        if(len(binaryNumber)==3):
            arr.append(0)
            arr.append(binaryNumber[0])
            arr.append(binaryNumber[1])
            arr.append(binaryNumber[2])
        if(len(binaryNumber)==2):
            arr.append(0)
            arr.append(0)
            arr.append(binaryNumber[0])
            arr.append(binaryNumber[1])
        if(len(binaryNumber)==1):
            arr.append(0)
            arr.append(0)
            arr.append(0)
            arr.append(binaryNumber[0])
            
    # for i in range(0, len(arr),4):
    #     print(arr[i],arr[i+1],arr[i+2],arr[i+3])
    return arr

In [85]:
def XOR(arr1, arr2):
    arr = []
    # print(len(arr1))
    # print(len(arr2))
    if(len(arr1)!=len(arr2)):
        return
    for i in range(len(arr1)):
        if(arr1[i]==arr2[i]):
            arr.append(0)
        else: 
            if(arr1[i] or arr2[i]):
                arr.append(1)
            else:
                arr.append(0)


    # for i in range(0, len(arr),6):
    #     print(arr[i],arr[i+1],arr[i+2],arr[i+3],arr[i+4],arr[i+5])

    # for i in range(0, len(arr),8):
    #     print(arr[i],arr[i+1],arr[i+2],arr[i+3],arr[i+4],arr[i+5],arr[i+6],arr[i+7])
    return arr

In [79]:
def roundKeyFunc():
    roundKey = []
    for i in range(RoundShift[roundNumber  -1]):
        C.append(C.pop(0)) #Circular Shift Left of Left Key
        D.append(D.pop(0)) #Circular Shift Left of Right Key
    for i in range(len(C)):
        roundKey.append(C[i])
    for i in range(len(D)):
        roundKey.append(D[i])

    roundKey = PC(roundKey,2)
    # for i in range(0, len(roundKey),6):
    #     print(roundKey[i],roundKey[i+1],roundKey[i+2],roundKey[i+3],roundKey[i+4],roundKey[i+5])
    return roundKey  
    
    

In [80]:
def roundPlainText():
    plainText = []
    for i in range(len(L)):
        plainText.append(L[i])
    for i in range(len(R)):
        plainText.append(R[i])
    for i in range(0, len(plainText),8):
            print(plainText[i],plainText[i+1],plainText[i+2],plainText[i+3],plainText[i+4],plainText[i+5],plainText[i+6],plainText[i+7])

In [81]:
#Permutation Choice 1 happens only once
Key = PC(K,1)
C = Key[:28]
D = Key[28:]
#Initial Permutation happens only once
Initial = IP(plainText)
L=Initial[:32]
R=Initial[32:]

In [83]:
while(roundNumber<=16):
    print(f"\nroundNumber:{roundNumber}")
    roundKey = roundKeyFunc() #gets key for round
    # roundKey = [0,0,0,0,0,0,0,1,  0,0,1,0,0,0,1,1,  0,1,0,0,0,1,0,1,  0,1,1,0,0,1,1,1,  1,0,0,0,1,0,0,1,  1,0,1,0,1,0,1,1]

    Expansion = EP(R)

    ExpansionXorKey = XOR(Expansion, roundKey)

    Sboxed = Sbox(ExpansionXorKey)

    Permutation = PF(Sboxed)

    temp = PermutationXorLeft = XOR(Permutation, L)
    L=[]
    for i in range(len(R)):
        L.append(R[i])
    R=[]
    for i in range(len(temp)):
        R.append(temp[i])
    
    plainText = roundPlainText()
    
    roundNumber+=1
# print(len(plainText),plainText)
# cipherText = inverseIP(plainText)


roundNumber:1
48
48
32
32
1 1 1 1 0 0 0 0
1 0 1 0 1 0 1 0
1 1 1 1 0 0 0 0
1 0 1 0 1 0 1 0
0 1 0 1 1 1 1 0
0 0 0 1 1 1 0 0
1 1 1 0 1 1 0 0
0 1 1 0 0 0 1 1

roundNumber:2
48
48
32
32
0 1 0 1 1 1 1 0
0 0 0 1 1 1 0 0
1 1 1 0 1 1 0 0
0 1 1 0 0 0 1 1
1 0 0 0 0 0 1 0
1 1 1 0 0 0 0 1
0 0 1 1 1 1 0 0
0 1 0 0 1 0 0 1

roundNumber:3
48
48
32
32
1 0 0 0 0 0 1 0
1 1 1 0 0 0 0 1
0 0 1 1 1 1 0 0
0 1 0 0 1 0 0 1
0 1 0 0 1 0 0 1
1 0 0 1 0 1 0 1
0 1 0 0 0 0 1 0
1 1 1 1 1 0 0 1

roundNumber:4
48
48
32
32
0 1 0 0 1 0 0 1
1 0 0 1 0 1 0 1
0 1 0 0 0 0 1 0
1 1 1 1 1 0 0 1
0 0 0 0 1 1 0 1
1 1 0 1 0 1 1 0
0 1 0 0 1 0 1 0
1 1 1 1 1 0 1 1

roundNumber:5
48
48
32
32
0 0 0 0 1 1 0 1
1 1 0 1 0 1 1 0
0 1 0 0 1 0 1 0
1 1 1 1 1 0 1 1
0 1 1 1 0 0 0 0
0 0 1 1 0 1 1 0
0 0 0 0 0 1 0 0
0 0 1 1 1 0 1 1

roundNumber:6
48
48
32
32
0 1 1 1 0 0 0 0
0 0 1 1 0 1 1 0
0 0 0 0 0 1 0 0
0 0 1 1 1 0 1 1
1 1 1 1 0 0 0 1
0 1 0 0 0 1 1 1
0 0 0 0 1 0 1 1
1 1 0 0 0 0 1 0

roundNumber:7
48
48
32
32
1 1 1 1 0 0 0 1
0 1 0 0 0 1 1 1
0 0 0 0 1 0

In [84]:
arr=L
for i in range(0, len(arr),8):
        print(arr[i],arr[i+1],arr[i+2],arr[i+3],arr[i+4],arr[i+5],arr[i+6],arr[i+7])
    

1 0 1 1 1 0 1 0
1 0 0 0 1 0 0 0
1 1 1 1 0 1 1 0
1 0 0 1 1 0 0 1
